In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\maxwe\Player and Team Ratings\NCAA\ncaa_mens_scores_2024.csv")

# Get a list of all unique teams
teams = pd.unique(df[['home_team', 'away_team']].values.ravel())

# Initialize ratings
ratings = pd.Series(1000, index=teams, dtype=float)

# Function to calculate expected score
def expected_score(rating_a, rating_b):
    return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))

# Rating parameters
K = 20
num_iterations = 10

# Iteratively update ratings
for _ in range(num_iterations):
    for _, row in df.iterrows():
        home = row['home_team']
        away = row['away_team']
        home_score = row['home_team_score']
        away_score = row['away_team_score']

        # Determine actual result
        if home_score > away_score:
            home_result = 1
        elif home_score < away_score:
            home_result = 0
        else:
            home_result = 0.5

        away_result = 1 - home_result

        # Expected results
        expected_home = expected_score(ratings[home], ratings[away])
        expected_away = expected_score(ratings[away], ratings[home])

        # Optional: add a small bonus for goal differential
        goal_diff = home_score - away_score
        margin_multiplier = np.log(abs(goal_diff) + 1) if goal_diff != 0 else 1

        # Update ratings
        ratings[home] += K * margin_multiplier * (home_result - expected_home)
        ratings[away] += K * margin_multiplier * (away_result - expected_away)

# Normalize ratings to 1-100 scale
min_rating = ratings.min()
max_rating = ratings.max()
normalized_ratings = ((ratings - min_rating) / (max_rating - min_rating)) * 99 + 1
normalized_ratings = normalized_ratings.round(2)

# Convert to DataFrame
rating_df = normalized_ratings.reset_index()
rating_df.columns = ['team', 'rating']
rating_df = rating_df.sort_values(by='rating', ascending=False)

# Show the ratings
rating_df.head(25)

,team,rating
134,Ohio St.,100.00
117,Denver,94.84
27,Vermont,93.59
82,Dayton,92.16
110,Marshall,92.03
56,Clemson,90.83
142,Pittsburgh,88.55
160,SMU,88.12
170,San Diego,85.15
171,Duke,85.01
